# Assignment 2

## Question 1

In [0]:
sc = spark.sparkContext

shakes = sc.textFile('s3://vcu-dapt-615/shakes.txt')

In [0]:
# Examine Data
shakes.take(num=50)

Out[2]: ['',
 'Project Gutenberg’s The Complete Works of William Shakespeare, by William',
 'Shakespeare',
 '',
 'This eBook is for the use of anyone anywhere in the United States and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. You may copy it, give it away or re-use it under the terms',
 'of the Project Gutenberg License included with this eBook or online at',
 'www.gutenberg.org. If you are not located in the United States, you’ll',
 'have to check the laws of the country where you are located before using',
 'this ebook.',
 '',
 '',
 'Title: The Complete Works of William Shakespeare',
 '',
 'Author: William Shakespeare',
 '',
 'Release Date: January 1994 [EBook #100]',
 'Last Updated: November 7, 2019',
 '',
 'Language: English',
 '',
 'Character set encoding: UTF-8',
 '',
 '*** START OF THIS PROJECT GUTENBERG EBOOK THE COMPLETE WORKS OF WILLIAM SHAKESPEARE ***',
 '',
 '',
 '',
 '',
 'The Complete Works of William Shakespeare',
 '',
 '',
 '',
 'by William Shakespeare',
 '',
 '',
 '',
 '',
 ' Contents',
 '',
 '',
 '',
 ' THE SONNETS',
 '',
 ' ALL’S WELL THAT ENDS WELL',
 '',
 ' THE TRAGEDY OF ANTONY AND CLEOPATRA',
 '',
 ' AS YOU LIKE IT',
 '']

In [0]:
# Create a remove punct function
import re
def removePunct(text):
    ans1 = text.lower()
    #re.sub(pattern, repl, string, count=0, flags=0)
    ans2 = re.sub('[^A-Za-z\s\d]', "", ans1)
    #print ans2
    ans3 = ans2.strip()
    return ans3

In [0]:
# remove punct from data
clean_shakes = shakes.map(removePunct)

In [0]:
# Check to make sure the function ran properly
clean_shakes.take(num=50)

Out[5]: ['',
 'project gutenbergs the complete works of william shakespeare by william',
 'shakespeare',
 '',
 'this ebook is for the use of anyone anywhere in the united states and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever you may copy it give it away or reuse it under the terms',
 'of the project gutenberg license included with this ebook or online at',
 'wwwgutenbergorg if you are not located in the united states youll',
 'have to check the laws of the country where you are located before using',
 'this ebook',
 '',
 '',
 'title the complete works of william shakespeare',
 '',
 'author william shakespeare',
 '',
 'release date january 1994 ebook 100',
 'last updated november 7 2019',
 '',
 'language english',
 '',
 'character set encoding utf8',
 '',
 'start of this project gutenberg ebook the complete works of william shakespeare',
 '',
 '',
 '',
 '',
 'the complete works of william shakespeare',
 '',
 '',
 '',
 'by william shakespeare',
 '',
 '',
 '',
 '',
 'contents',
 '',
 '',
 '',
 'the sonnets',
 '',
 'alls well that ends well',
 '',
 'the tragedy of antony and cleopatra',
 '',
 'as you like it',
 '']

In [0]:
# create a list of words
shakespeareWordsRDD = clean_shakes.flatMap(lambda line: line.split(" "))

In [0]:
# count words
shakespeareWordCount = shakespeareWordsRDD.count()

In [0]:
shakespeareWordCount

Out[8]: 1006175

In [0]:
# get rid of spaces
shakeWordsRDD = shakespeareWordsRDD.filter(lambda x: x!="")

In [0]:
# create a function to count indiv words
def wordCount(wordListRDD):
    """Creates a pair RDD with word counts from an RDD of words.

    Args:
        wordListRDD (RDD of str): An RDD consisting of words.

    Returns:
        RDD of (str, int): An RDD consisting of (word, count) tuples.
    """
    #<FILL IN>
    answer = wordListRDD.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
    return answer

In [0]:
# create a variable of a word counts
WordsAndCounts = wordCount(shakeWordsRDD)

In [0]:
WordsAndCounts.take(num=50)

Out[12]: [('project', 107),
 ('of', 18811),
 ('shakespeare', 13),
 ('this', 7177),
 ('ebook', 14),
 ('is', 9713),
 ('use', 361),
 ('anyone', 11),
 ('anywhere', 9),
 ('in', 12186),
 ('united', 20),
 ('other', 725),
 ('world', 679),
 ('at', 2744),
 ('no', 4075),
 ('restrictions', 2),
 ('whatsoever', 17),
 ('may', 1806),
 ('give', 1419),
 ('away', 911),
 ('reuse', 2),
 ('gutenberg', 36),
 ('online', 4),
 ('are', 3726),
 ('have', 6267),
 ('check', 32),
 ('country', 163),
 ('where', 1367),
 ('before', 943),
 ('using', 14),
 ('title', 97),
 ('january', 3),
 ('1994', 1),
 ('100', 3),
 ('last', 301),
 ('2019', 1),
 ('language', 44),
 ('set', 508),
 ('encoding', 1),
 ('start', 39),
 ('contents', 49),
 ('alls', 44),
 ('ends', 55),
 ('tragedy', 26),
 ('cleopatra', 264),
 ('as', 6176),
 ('like', 1929),
 ('comedy', 12),
 ('errors', 15),
 ('prince', 776)]

In [0]:
WordsAndCounts.count()

Out[56]: 31512

## Answer 1

In [0]:
# Answer 1
WordsAndCounts.filter(lambda x: x[0] in ['wash','foot','table']).collect()

Out[64]: [('table', 55), ('foot', 177), ('wash', 55)]

## Question 2

In [0]:
reviews = spark.read.parquet('s3://amazon-reviews-pds/parquet/')
reviews.createOrReplaceTempView('reviews')

In [0]:
spark.sql('SELECT count(*) FROM reviews').show()

+---------+
 count(1)|
+---------+
160796570|
+---------+

### Count of unique product IDs

In [0]:
spark.sql('SELECT COUNT(DISTINCT product_id) Unique_Product_IDs FROM reviews').show()

+--------------------------+
count(DISTINCT product_id)|
+--------------------------+
 21460962|
+--------------------------+

### Product with most reviews and count

In [0]:
spark.sql('SELECT product_id, COUNT(DISTINCT review_id) Total_Reviews FROM reviews GROUP BY product_id ORDER BY Total_Reviews DESC LIMIT 1').show()

+----------+-------------+
product_id|Total_Reviews|
+----------+-------------+
B00FAPF5U0| 59492|
+----------+-------------+

### Product title with most reviews

In [0]:
spark.sql('SELECT product_id, product_title, COUNT(DISTINCT review_id) Total_Reviews FROM reviews GROUP BY product_id, product_title ORDER BY Total_Reviews DESC LIMIT 1').show()

+----------+----------------+-------------+
product_id| product_title|Total_Reviews|
+----------+----------------+-------------+
B00FAPF5U0|Candy Crush Saga| 59492|
+----------+----------------+-------------+

### Average star rating for all reviews
* Count of below average ratings 
* Count of above average ratings

In [0]:
spark.sql('SELECT ROUND(AVG(star_rating),2) Overall_Avg_Star_Rating FROM reviews').show()

+-----------------------+
Overall_Avg_Star_Rating|
+-----------------------+
 4.21|
+-----------------------+

In [0]:
spark.sql('SELECT COUNT(DISTINCT review_id) Total_Reviews_Below_Avg  FROM reviews WHERE star_rating < (SELECT ROUND(AVG(star_rating),2) Overall_Avg_Star_Rating FROM reviews)').show()

+-----------------------+
Total_Reviews_Below_Avg|
+-----------------------+
 58776421|
+-----------------------+

In [0]:
spark.sql('SELECT COUNT(DISTINCT review_id) Total_Reviews_Above_Avg FROM reviews WHERE star_rating > (SELECT ROUND(AVG(star_rating),2) Overall_Avg_Star_Rating FROM reviews)').show()

+-----------------------+
Total_Reviews_Above_Avg|
+-----------------------+
 95088983|
+-----------------------+

### Highest average rating (5)

In [0]:
spark.sql('SELECT product_title, ROUND(AVG(star_rating),2) Avg_Star_Rating FROM reviews GROUP BY product_title ORDER BY Avg_Star_Rating DESC LIMIT 5').show()

+--------------------+---------------+
 product_title|Avg_Star_Rating|
+--------------------+---------------+
Hope and Courage ...| 5.0|
Sai Rapture: The ...| 5.0|
Job Quest: How to...| 5.0|
 Forty Wolves| 5.0|
Germaine Moreau a...| 5.0|
+--------------------+---------------+

### Count of title containing 'candy' and review text containing 'awful'

In [0]:
from pyspark.sql.types import BooleanType
def contains_word(x, word):
 try:
   return word in str(x).lower()
 except:
   return False
spark.udf.register("contains_word", contains_word, BooleanType()) 

Out[92]: <function __main__.contains_word(x, word)>

In [0]:
spark.sql('SELECT COUNT(DISTINCT product_title) Candy_Title_Count FROM reviews WHERE contains_word(product_title, "candy") = TRUE').show()

+-----------------+
Candy_Title_Count|
+-----------------+
 36314|
+-----------------+

In [0]:
spark.sql('SELECT COUNT(DISTINCT review_id) Awful_Review_Count FROM reviews WHERE contains_word(review_body, "awful") = TRUE').show()

+------------------+
Awful_Review_Count|
+------------------+
 495670|
+------------------+